In [2]:
import pandas as pd
import numpy as np
import scipy.stats as scipy

In [3]:
df = pd.read_csv("./Data/E0_13.csv")
df_14 = pd.read_csv("./Data/E0_14.csv")

In [4]:
df.columns

Index([u'Div', u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR',
       u'HTHG', u'HTAG', u'HTR', u'Referee', u'HS', u'AS', u'HST', u'AST',
       u'HF', u'AF', u'HC', u'AC', u'HY', u'AY', u'HR', u'AR', u'B365H',
       u'B365D', u'B365A', u'BWH', u'BWD', u'BWA', u'IWH', u'IWD', u'IWA',
       u'LBH', u'LBD', u'LBA', u'PSH', u'PSD', u'PSA', u'WHH', u'WHD', u'WHA',
       u'SJH', u'SJD', u'SJA', u'VCH', u'VCD', u'VCA', u'Bb1X2', u'BbMxH',
       u'BbAvH', u'BbMxD', u'BbAvD', u'BbMxA', u'BbAvA', u'BbOU', u'BbMx>2.5',
       u'BbAv>2.5', u'BbMx<2.5', u'BbAv<2.5', u'BbAH', u'BbAHh', u'BbMxAHH',
       u'BbAvAHH', u'BbMxAHA', u'BbAvAHA', u'PSCH', u'PSCD', u'PSCA'],
      dtype='object')

In [5]:
res_13 = df.ix[:,:23]
res_13 = res_13.drop(['Div','Date','Referee'],axis=1)
res_14 = df_14.ix[:,:23]
res_14 = res_14.drop(['Div','Date','Referee'],axis=1)

bet_13 = df.ix[:,23:]

In [6]:
res_13.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,Arsenal,Aston Villa,1,3,A,1,1,D,16,9,4,4,15,18,4,3,4,5,1,0
1,Liverpool,Stoke,1,0,H,1,0,H,26,10,11,4,11,11,12,6,1,1,0,0
2,Norwich,Everton,2,2,D,0,0,D,8,19,2,6,13,10,6,8,2,0,0,0
3,Sunderland,Fulham,0,1,A,0,0,D,20,5,3,1,14,14,6,1,0,3,0,0
4,Swansea,Man United,1,4,A,0,2,A,17,15,6,7,13,10,7,4,1,3,0,0


In [7]:
#Team, Home Goals Score, Away Goals Score, Attack Strength, Home Goals Conceded, Away Goals Conceded, Defensive Strength
table_13 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS'))

In [8]:
avg_home_scored_13 = res_13.FTHG.sum() / 380.0
avg_away_scored_13 = res_13.FTAG.sum() / 380.0
avg_home_conceded_13 = avg_away_scored_13
avg_away_conceded_13 = avg_home_scored_13
print "Average number of goals at home",avg_home_scored_13
print "Average number of goals away", avg_away_scored_13
print "Average number of goals conceded at home",avg_away_conceded_13
print "Average number of goals conceded away",avg_home_conceded_13


Average number of goals at home 1.57368421053
Average number of goals away 1.19473684211
Average number of goals conceded at home 1.57368421053
Average number of goals conceded away 1.19473684211


In [9]:
res_home = res_13.groupby('HomeTeam')
res_away = res_13.groupby('AwayTeam')

In [10]:
list(res_home)[0]

('Arsenal',
     HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  HS  AS  HST  \
 0    Arsenal     Aston Villa     1     3   A     1     1   D  16   9    4   
 27   Arsenal       Tottenham     1     0   H     1     0   H  12  14    5   
 46   Arsenal           Stoke     3     1   H     2     1   H  16   9    8   
 70   Arsenal         Norwich     4     1   H     1     0   H  20  12   11   
 90   Arsenal       Liverpool     2     0   H     1     0   H  12  12    7   
 110  Arsenal     Southampton     2     0   H     1     0   H   9  10    4   
 131  Arsenal            Hull     2     0   H     1     0   H  20   7    7   
 147  Arsenal         Everton     1     1   D     0     0   D  11  12    5   
 169  Arsenal         Chelsea     0     0   D     0     0   D   7  13    2   
 190  Arsenal         Cardiff     2     0   H     0     0   D  28   8    6   
 210  Arsenal          Fulham     2     0   H     0     0   D  22   8    8   
 237  Arsenal  Crystal Palace     2     0   H     0 

In [11]:
table_13.Team = res_home.HomeTeam.all().values
table_13.HGS = res_home.FTHG.sum().values
table_13.HGC = res_home.FTAG.sum().values
table_13.AGS = res_away.FTAG.sum().values
table_13.AGC = res_away.FTHG.sum().values
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,36,32,NaN,NaN,11,30,NaN,NaN
1,Aston Villa,22,17,NaN,NaN,29,32,NaN,NaN
2,Cardiff,20,12,NaN,NaN,35,39,NaN,NaN
3,Chelsea,43,28,NaN,NaN,11,16,NaN,NaN
4,Crystal Palace,18,15,NaN,NaN,23,25,NaN,NaN


In [12]:
table_13.HAS = (table_13.HGS / 19.0) / avg_home_scored_13
table_13.AAS = (table_13.AGS / 19.0) / avg_away_scored_13
table_13.HDS = (table_13.HGC / 19.0) / avg_home_conceded_13
table_13.ADS = (table_13.AGC / 19.0) / avg_away_conceded_13
table_13

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,36,32,1.204013,1.409692,11,30,0.484581,1.003344
1,Aston Villa,22,17,0.735786,0.748899,29,32,1.277533,1.070234
2,Cardiff,20,12,0.668896,0.528634,35,39,1.541850,1.304348
3,Chelsea,43,28,1.438127,1.233480,11,16,0.484581,0.535117
4,Crystal Palace,18,15,0.602007,0.660793,23,25,1.013216,0.836120
5,Everton,38,23,1.270903,1.013216,19,20,0.837004,0.668896
6,Fulham,24,16,0.802676,0.704846,38,47,1.674009,1.571906
7,Hull,20,18,0.668896,0.792952,21,32,0.925110,1.070234
8,Liverpool,53,48,1.772575,2.114537,18,32,0.792952,1.070234
9,Man City,63,39,2.107023,1.718062,13,24,0.572687,0.802676


In [13]:
def exp_goals(mean):
    max_pmf = 0;
    for i in xrange(7):
        pmf = scipy.distributions.poisson.pmf(i,mean) * 100 
        if pmf > max_pmf:
            max_pmf = pmf
            goals = i
    return goals

In [14]:
test_13 = res_13.ix[:,0:5]
test_13.head()
test_14 = res_14.ix[:,0:5]
test_14.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Arsenal,Crystal Palace,2.0,1.0,H
1,Leicester,Everton,2.0,2.0,D
2,Man United,Swansea,1.0,2.0,A
3,QPR,Hull,0.0,1.0,A
4,Stoke,Aston Villa,0.0,1.0,A


In [15]:
table_13[table_13['Team'] == 'Arsenal']
test_14['ER'] = ''
test_14 = test_14.drop(test_14.index[380],axis=0)

In [16]:
results = []
for index, row in test_14.iterrows():

    home_team = table_13[table_13['Team'] == row['HomeTeam']]
    away_team = table_13[table_13['Team'] == row['AwayTeam']]
    #print "Home : ", home_team.HAS.values, "Away: ", away_team.AAS.
    if row.HomeTeam not in ['Leicester', 'QPR', 'Burnley'] and row.AwayTeam not in ['Leicester', 'QPR', 'Burnley']:
        EH = home_team.HAS.values * away_team.ADS.values * avg_home_scored_13
        EA = home_team.HDS.values * away_team.AAS.values * avg_home_conceded_13
        #print row.HomeTeam, row.AwayTeam
        if EH[0] > EA[0]:
            results.append('H')
        else:
            results.append('A')
    else:
        results.append('D')

In [17]:
test_14['ER'] = results

In [18]:
from sklearn.metrics import accuracy_score
test_14

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,ER
0,Arsenal,Crystal Palace,2.0,1.0,H,H
1,Leicester,Everton,2.0,2.0,D,D
2,Man United,Swansea,1.0,2.0,A,H
3,QPR,Hull,0.0,1.0,A,D
4,Stoke,Aston Villa,0.0,1.0,A,H
5,West Brom,Sunderland,2.0,2.0,D,H
6,West Ham,Tottenham,0.0,1.0,A,A
7,Liverpool,Southampton,2.0,1.0,H,H
8,Newcastle,Man City,0.0,2.0,A,A
9,Burnley,Chelsea,1.0,3.0,A,D


In [19]:
accuracy_score(test_14['FTR'],test_14['ER'])

0.4263157894736842